In [1]:
from itertools import product
from itertools import combinations



import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader



import torch.nn.functional as F
#from torcheval.metrics.functional import multiclass_confusion_matrix
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

from torchvision.utils import save_image
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torchvision import transforms



from itertools import product
from itertools import combinations


import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation ,FFMpegWriter ,PillowWriter

import dill
import numpy as np
import os
import json
import pandas as pd
from tqdm import tqdm
import copy

In [2]:
torch.set_default_dtype(torch.float16)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Trainloader code

In [3]:
sorted([ [2,3,7], [9,1,4] ])

[[2, 3, 7], [9, 1, 4]]

In [4]:
Pairs_exp=list()
All=list(range(10))
progress_bar = tqdm(
            range(0, 36*23436),
            initial=0,
            desc="Steps",
        )
for act in range(6):
    for ep in range(6):
        for sample in range(2,9,1):
            S=combinations(range(10), sample)
            #All=list(range(10))
            for i in S :
                L1=list(i)
                L2=[k for k in All if k not in L1]
                for sample2 in range(2,9,1):
                    S2=combinations(L2, sample2)
                    for j in S2:
                        pair=[]
                        pair.append(L1)
                        sub=list(j)
                        pair.append(sub)
                        pair.sort()
                        pair.append(act)
                        pair.append(ep)
                        if pair not in Pairs_exp :
                            Pairs_exp.append(pair)
                            progress_bar.update(1)
print(len(Pairs_exp))
train_pair=[]
train_tgt=[]

val_pair=[]
val_tgt=[]

test_pair=[]
test_tgt=[]

for pair in tqdm(Pairs_exp):
    if (len(pair[0])==5 and len(pair[1])==5) or(len(pair[0])==4 and len(pair[1])==4) or(len(pair[0])==3 and len(pair[1])==3) or (len(pair[0])==2 and len(pair[1])==2):
        test_pair.append(pair)
        tg=(pair[0]+pair[1]).sort()
        test_tgt.append(tg)
    elif (len(pair[0])==7 and len(pair[1])==2) or (len(pair[0])==2 and len(pair[1])==7) or (len(pair[0])==6 and len(pair[1])==3) or (len(pair[0])==3 and len(pair[1])==6) or (len(pair[0])==4 and len(pair[1])==5) or (len(pair[0])==5 and len(pair[1])==4):
        val_pair.append(pair)
        tg=(pair[0]+pair[1]).sort()
        val_tgt.append(tg)
    else :
        train_pair.append(pair)
        tg=(pair[0]+pair[1]).sort()
        train_tgt.append(tg)

Steps: 100%|██████████| 843696/843696 [4:03:49<00:00,  2.15it/s]     

843696



100%|██████████| 843696/843696 [00:00<00:00, 878512.22it/s]


In [5]:
import random
random.seed(881)
print(train_pair[:10])
random.shuffle(test_pair)
random.shuffle(train_pair)
random.shuffle(val_pair)
print(train_pair[:10])

[[[0, 1], [2, 3, 4], 0, 0], [[0, 1], [2, 3, 5], 0, 0], [[0, 1], [2, 3, 6], 0, 0], [[0, 1], [2, 3, 7], 0, 0], [[0, 1], [2, 3, 8], 0, 0], [[0, 1], [2, 3, 9], 0, 0], [[0, 1], [2, 4, 5], 0, 0], [[0, 1], [2, 4, 6], 0, 0], [[0, 1], [2, 4, 7], 0, 0], [[0, 1], [2, 4, 8], 0, 0]]
[[[0, 2, 6], [1, 5, 7, 9], 0, 2], [[1, 2, 3, 5], [6, 7, 9], 3, 0], [[0, 8, 9], [2, 3, 4, 5, 6], 4, 1], [[2, 3, 5], [4, 8], 4, 1], [[0, 1, 2, 5, 7], [3, 4], 3, 4], [[0, 1, 3, 4, 5, 6, 7, 9], [2, 8], 1, 3], [[0, 1, 2, 4, 5, 6, 7], [3, 8, 9], 3, 5], [[1, 8, 9], [2, 3, 6, 7], 4, 5], [[0, 3], [1, 4, 6, 8, 9], 2, 2], [[1, 4], [2, 3, 5, 6], 5, 3]]


In [4]:
np.save('./train_pair_seed881.npy', train_pair)
np.save('./val_pair_seed881.npy', val_pair)
np.save('./test_pair_seed881.npy', test_pair)


NameError: name 'train_pair' is not defined

In [7]:
test_pair[:10]

[[[2, 4, 6], [5, 7, 9], 3, 5],
 [[1, 5], [3, 4], 0, 5],
 [[1, 8, 9], [2, 5, 6], 5, 1],
 [[1, 5, 8], [4, 7, 9], 1, 4],
 [[0, 1, 4, 9], [2, 5, 6, 8], 4, 0],
 [[0, 3, 8], [2, 7, 9], 5, 0],
 [[0, 2, 4, 6], [3, 5, 8, 9], 1, 0],
 [[0, 1, 6, 7], [3, 4, 5, 9], 1, 3],
 [[0, 5, 8, 9], [1, 4, 6, 7], 3, 0],
 [[0, 1, 3], [2, 7, 9], 4, 5]]

In [5]:

# Load the list
train_pair = np.load('train_pair_seed881.npy', allow_pickle=True)
test_pair = np.load('test_pair_seed881.npy', allow_pickle=True)
val_pair = np.load('val_pair_seed881.npy', allow_pickle=True)

"""print(loaded_list[:10])
test_pair[0]"""

'print(loaded_list[:10])\ntest_pair[0]'

In [6]:
def batchify(lst, batch_size):
    return [lst[i:i+batch_size] for i in range(0, len(lst), batch_size)]

# Example usage:
my_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
batch_size = 4
batches = batchify(my_list, batch_size)
print(batches)

[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10]]


In [7]:
from torch.utils.data import TensorDataset, ConcatDataset, DataLoader

import random
import ast
        
class CustomDataset(TensorDataset): #L_activations=["gelu","relu","silu","leakyrelu","sigmoid","tanh"]
    def __init__(self,L_exp,batch_size=300,df_path="./data/Merged zoo.csv"):

        self.df_path = df_path
        self.df=pd.read_csv(df_path)
        self.L_exp=L_exp
        self.params_cols=list(self.df.columns[17:-2])
        def batchify(lst, batch_size):
            return [lst[i:i+batch_size] for i in range(0, len(lst), batch_size)]

        self.batchs=batchify(self.L_exp, batch_size)
        self.D_activ={'0':"gelu",'1':"relu",'2':"silu",'3':"leakyrelu",'4':"sigmoid",'5':"tanh"}
        self.D_epoch={'0':"36",'1':"31",'2':"21",'3':"26",'4':"11",'5':"16"}

    def __len__(self):
        return len(self.L_exp) #num_sublists #len(self.exp)
    
    
    def __getitem__(self, idx):
        
        batch=self.batchs[idx]
        L_Stream1=[]
        L_Stream2=[]
        tgt=[]
        
        L_Exp=[]
        L_ACC=[]
        L_indexes=[]
        
        for i in range(len(batch)):
                rowk=self.df[(self.df["label"]=='{}'.format(batch[i][0]))&(self.df["epoch"]==int(self.D_epoch[str(batch[i][2])]))&(self.df[self.D_activ[str(batch[i][3])]]==float(1))]
                L_Stream1.append(torch.from_numpy(rowk[self.params_cols].to_numpy().astype('float16')))
                ind1=int(rowk.index[0])
                ACC1=self.df.loc[ind1]["Accuracy"]

                

                
                rowk=self.df[(self.df["label"]=='{}'.format(batch[i][1]))&(self.df["epoch"]==int(self.D_epoch[str(batch[i][2])]))&(self.df[self.D_activ[str(batch[i][3])]]==float(1))]
                L_Stream2.append(torch.from_numpy(rowk[self.params_cols].to_numpy().astype('float16')))
                ind2=int(rowk.index[0])
                ACC2=self.df.loc[ind2]["Accuracy"]
                
                
                tg=batch[i][0]+batch[i][1]
                tg.sort()
                rowk=self.df[(self.df["label"]=='{}'.format(tg))&(self.df["epoch"]==int(self.D_epoch[str(batch[i][2])]))&(self.df[self.D_activ[str(batch[i][3])]]==float(1))]
                ind3=int(rowk.index[0])
                tgt.append(torch.from_numpy(rowk[self.params_cols].to_numpy().astype('float16')))
                ACC3=float(rowk["Accuracy"].values)
                

                L_ACC.append([ACC1,ACC2,ACC3])
                L_indexes.append([ind1,ind2,ind3])

        Stream1=torch.stack(L_Stream1).squeeze()
        Stream2=torch.stack(L_Stream2).squeeze()
        target=torch.stack(tgt).squeeze()
        
        #Stream2=Stream2.reshape((int(Stream2.shape[0]),1, int(Stream2.shape[1])))
        #target=target.reshape((target.shape[0],1, target.shape[1]))
        #print(Stream1.shape,Stream2.shape,target.shape)
        
        loaded = torch.stack([Stream1,Stream2,target],dim=1)
        
        ACC=L_ACC
        batch_indices=L_indexes
        artifacts= loaded,batch,ACC,batch_indices

        return artifacts



In [8]:
cs_tr=CustomDataset(train_pair,batch_size=512)

In [9]:
Dataset,EXP,ACC,U = cs_tr[0]
x1,x2,tg = Dataset[:,0,:], Dataset[:,1,:],Dataset[:,2,:]

/tmp/ipykernel_1758842/2203921738.py:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ACC3=float(rowk["Accuracy"].values)
/tmp/ipykernel_1758842/2203921738.py:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ACC3=float(rowk["Accuracy"].values)
/tmp/ipykernel_1758842/2203921738.py:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ACC3=float(rowk["Accuracy"].values)
/tmp/ipykernel_1758842/2203921738.py:55: DeprecationWarning: Conversion of an array with ndim > 0 to

In [10]:
x1.shape

torch.Size([512, 2464])

In [11]:
cs_tr.df.iloc[:5,:12]

,label,0,1,2,3,4,5,6,7,8,9,gelu
0,"[0, 1]",1,1,0,0,0,0,0,0,0,0,1.0
1,"[0, 2]",1,0,1,0,0,0,0,0,0,0,1.0
2,"[0, 3]",1,0,0,1,0,0,0,0,0,0,1.0
3,"[0, 4]",1,0,0,0,1,0,0,0,0,0,1.0
4,"[0, 5]",1,0,0,0,0,1,0,0,0,0,1.0


# Autoencoder

In [12]:
#!pip install einops

In [13]:
import torch
import torch.nn as nn
import numpy as np



import itertools
from einops import repeat

import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable


# # Transformer Shared Layers


def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len=80,device="cuda"): #d"
        super().__init__()
        self.d_model = d_model
        self.device = device

        # create constant 'pe' matrix with values dependant on
        # pos and i
        self.pe = self._generate_positional_encoding(max_seq_len, d_model)

    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.d_model)
        # add constant to embedding
        seq_len = x.size(1)
        # dynamically adjust positional encoding matrix based on sequence length
        pe = self.pe[:, :seq_len]
        pe=pe.to(self.device)
        x=x.to(self.device)
        x = x + pe
        return x

    def _generate_positional_encoding(self, max_seq_len, d_model):
        pe = torch.zeros(max_seq_len, d_model)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        return pe

In [14]:
def attention(q, k, v, d_k, mask=None, dropout=None):
    scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        mask = mask.unsqueeze(1)
        scores = scores.masked_fill(mask == 0, -1e9)
    scores = F.softmax(scores, dim=-1)

    if dropout is not None:
        scores = dropout(scores)

    output = torch.matmul(scores, v)

    return output, scores





class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout=0.1):
        super().__init__()

        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads

        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):

        bs = q.size(0)

        # perform linear operation and split into h heads

        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)

        # transpose to get dimensions bs * h * sl * d_model

        k = k.transpose(1, 2)
        q = q.transpose(1, 2)
        v = v.transpose(1, 2)
        # calculate attention using function we will define next
        scores, sc = attention(q, k, v, self.d_k, mask, self.dropout)

        # concatenate heads and put through final linear layer
        concat = scores.transpose(1, 2).contiguous().view(bs, -1, self.d_model)

        output = self.out(concat)

        return output, sc


class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, normalize=True, dropout=0.1, d_ff=2048):
        super().__init__()
        self.normalize = normalize
        if normalize:
            self.norm_1 = Norm(d_model)
            self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model, dropout=dropout)
        self.ff = FeedForward(d_model, d_ff=d_ff, dropout=dropout)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

    def forward(self, x, mask):
        if self.normalize:
            x2 = self.norm_1(x)
        else:
            x2 = x.clone()
        res, sc = self.attn(x2, x2, x2, mask)
        # x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x = x + self.dropout_1(res)
        if self.normalize:
            x2 = self.norm_2(x)
        else:
            x2 = x.clone()
        x = x + self.dropout_2(self.ff(x2))
        # return x
        return x, sc

In [15]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout=0.1):
        super().__init__()
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x


class Norm(nn.Module):
    def __init__(self, d_model, eps=1e-6):
        super().__init__()

        self.size = d_model
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps

    def forward(self, x):
        norm = (
            self.alpha
            * (x - x.mean(dim=-1, keepdim=True))
            / (x.std(dim=-1, keepdim=True) + self.eps)
            + self.bias
        )
        return norm

In [16]:
class EmbedderNeuronGroup(nn.Module):
    def __init__(self, d_model, seed=22):
        super().__init__()
        #print("EmbedderNeuroneGroup")
        self.neuron_l1 = nn.Linear(200, d_model) #8
        self.neuron_l2 = nn.Linear(72, d_model) #12

    def forward(self, x):
        return self.multiLinear(x)

    def multiLinear(self, v):
        #print("multi-linear method",v.shape)

        l = []

        for ndx in range(8):
            idx_start = ndx * 200
            idx_end = idx_start + 200
            l.append(self.neuron_l1(v[:,idx_start:idx_end]))

        # l2
        for ndx in range(12):
            idx_start = 200*8 + ndx * 72
            idx_end = idx_start + 72
            l.append(self.neuron_l2(v[:,idx_start:idx_end]))
        #print(len(l))
        #print(len(l[0]))
        final = torch.stack(l, dim=1)

        # print(final.shape)
        return final



In [17]:
class EncoderNeuronGroup(nn.Module):
    def __init__(self, d_model, N, heads, max_seq_len, dropout, d_ff):
        super().__init__()
        self.N = N
        self.embed = EmbedderNeuronGroup(d_model)
        self.pe = PositionalEncoder(d_model, max_seq_len)
        print("encoder droupout init",dropout)
        self.layers = get_clones(EncoderLayer(d_model, heads, normalize=True,dropout=dropout, d_ff=d_ff), N)
        self.norm = Norm(d_model)

    def forward(self, src, mask=None):
        scores = []
        x = self.embed(src)
        x = self.pe(x)
        for i in range(self.N):
            self.layers[i] = self.layers[i].to(device)
            x, sc = self.layers[i](x, mask)
            scores.append(sc)
        return self.norm(x), scores

In [18]:
# max_seq_len=176,
# N=4
# heads=3
# d_model=900
# d_ff=900
# neck=700
# dropout=0.1
# # Enc=EncoderNeuronGroup(d_model=d_model, N=N, heads=heads, max_seq_len=max_seq_len, dropout=dropout,d_ff=d_ff)
# # vec1 = torch.rand(1,2464)
# # res,scores=Enc(vec1)
# # res.shape

In [19]:
# vec2neck = nn.Linear(d_ff*2, neck)
# print(res.shape)
# out3=torch.cat([res,res], dim=2)
# print("neck input:",out3.shape)
# sum_r=torch.sum(out3, dim=1, keepdim=False)
# vec2=vec2neck(sum_r)
# print(len(vec2))
# tanh = nn.Tanh()
# neck_t=tanh(vec2)
# print("neck shape:",neck_t.shape)

In [20]:
class Seq2Vec(nn.Module):
    def __init__(self, d_model, max_seq_len):
        super().__init__()
        self.d_model = d_model
        self.max_seq_len = max_seq_len

        # Define linear layers
        self.linear = nn.Linear(max_seq_len * d_model, 2464)

    def forward(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, -1)  # Flatten the sequence dimension
        x = self.linear(x)
        return x

class Neck2Seq(nn.Module):
    def __init__(self, d_model, neck,max_seq_length):
        super().__init__()
        self.neurons = nn.ModuleList([nn.Linear(neck, d_model) for _ in range(max_seq_length)])
    def forward(self, x):
        l = [neuron(x) for neuron in self.neurons]
        final = torch.stack(l, dim=1)
        return final
class DecoderNeuronGroup(nn.Module):
    def __init__(self, d_model, N, heads, max_seq_len, dropout, d_ff, neck):
        super().__init__()
        self.N = N
        self.embed = Neck2Seq(d_model, neck,max_seq_len)
        self.pe = PositionalEncoder(d_model, max_seq_len)
        print("decoder droupout init",dropout)
        self.layers = get_clones(EncoderLayer(d_model, heads,normalize=True,dropout=dropout, d_ff=d_ff), N)
        self.norm = Norm(d_model)
        self.lay = Seq2Vec(d_model=d_model,max_seq_len=max_seq_len)

    def forward(self, src, mask=None):
        scores = []
        x = self.embed(src)
        x = self.pe(x)
        for i in range(self.N):
            x, sc = self.layers[i](x, mask)
            scores.append(sc)
        return self.lay(self.norm(x)), scores

In [21]:
# Dec=DecoderNeuronGroup(d_model=d_model, N=N, heads=heads, max_seq_len=max_seq_len, dropout=dropout,d_ff=d_ff,neck=neck)
# res,scores=Dec(neck_t)
# res.shape

In [22]:
from collections import OrderedDict
import copy
import numpy as np

class TransformerAE(nn.Module):
    def __init__(
        self,
        max_seq_len=10,
        N=1,
        heads=1,
        d_model=100,
        d_ff=100,
        neck=20,
        dropout=0.1,
        **kwargs,):
        super().__init__()
        self.N=N
        self.heads=heads
        self.dropout=dropout
        self.d_ff=d_ff
        self.d_model=d_model
        self.max_seq_len=max_seq_len
        self.neck=neck


        self.enc1 = EncoderNeuronGroup(d_model=self.d_model, N=self.N, heads=self.heads, max_seq_len=self.max_seq_len, dropout=self.dropout,d_ff=self.d_ff)
        self.enc2 = EncoderNeuronGroup(d_model=self.d_model, N=self.N, heads=self.heads, max_seq_len=self.max_seq_len, dropout=self.dropout,d_ff=self.d_ff)
        self.dec = DecoderNeuronGroup(d_model=self.d_model, N=self.N, heads=self.heads, max_seq_len=self.max_seq_len, dropout=self.dropout,d_ff=self.d_ff,neck=self.neck)
        # Addition Approach
        #print("Addition Approach!")
        self.vec2neck = nn.Linear(self.d_ff*2, self.neck)
        # Stacking Approach
        #print("Stack Approach!")
        #self.vec2neck = nn.Linear(2*self.d_ff * self.max_seq_len, self.neck)

        self.tanh = nn.Tanh()

        # Xavier Uniform Initialitzation
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
                
        if torch.cuda.is_available():
            self.cuda()

    def forward(self, inp1,inp2):

        # First Approach
        out1, scEnc1 = self.enc1(inp1)
        #print("Encoder 1 shape:",out1.shape)
        out2, scEnc2 = self.enc2(inp2)
        #print("Encoder 2 shape:",out2.shape)
        out3=torch.cat([out1,out2], dim=2)

        #print("neck input:",out3.shape)
        sum_r=torch.sum(out3, dim=1, keepdim=False)
        vec2=self.vec2neck(sum_r)
        #print(len(vec2))
        tanh = nn.Tanh()
        neck_t=tanh(vec2)
        #print("neck shape:",neck_t.shape)

        out, scDec = self.dec(neck_t)
        #print("decoder shape:",out.shape)
        return out, neck_t, scEnc1,scEnc2, scDec

    def count_parameters(self, model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

    def numParams(self):
        encNumParams = self.count_parameters(self.enc1)
        neckNumParams = self.count_parameters(self.vec2neck)
        decNumParams = self.count_parameters(self.dec)
        modelParams = self.count_parameters(self)

        return (
            "EncParams: {}, NeckParams: {}, DecParams: {}, || ModelParams: {} ".format(
                encNumParams, neckNumParams, decNumParams, modelParams
            )
        )

In [23]:
len(test_pair),len(train_pair),len(val_pair)#,len(test_tgt),len(train_tgt),len(val_tgt)

(159516, 595620, 88560)

In [24]:

mod = TransformerAE(max_seq_len=20,
                    N=4,
                    heads=4,
                    d_model=300,
                    d_ff=300,
                    neck=50,
                    dropout=0.12
                   )

encoder droupout init 0.12
encoder droupout init 0.12
decoder droupout init 0.12


In [25]:


print(mod.numParams())
#x1 = torch.rand(1,2464)
#x2 = torch.rand(1,2464)
mod=mod.to(device).to(torch.float16)

#x1=x1.to(torch.float32)
#x2=x2.to(torch.float32)
x1=x1.to(device)
x2=x2.to(device)
mod=mod.to(device)
out = mod(x1,x2)
print("Output Shape: ", out[0].shape)

from torchinfo import summary

summary(mod)


EncParams: 2254800, NeckParams: 30050, DecParams: 17265064, || ModelParams: 21804714 
Output Shape:  torch.Size([512, 2464])


Layer (type:depth-idx)                        Param #
TransformerAE                                 --
├─EncoderNeuronGroup: 1-1                     --
│    └─EmbedderNeuronGroup: 2-1               --
│    │    └─Linear: 3-1                       60,300
│    │    └─Linear: 3-2                       21,900
│    └─PositionalEncoder: 2-2                 --
│    └─ModuleList: 2-3                        --
│    │    └─EncoderLayer: 3-3                 543,000
│    │    └─EncoderLayer: 3-4                 543,000
│    │    └─EncoderLayer: 3-5                 543,000
│    │    └─EncoderLayer: 3-6                 543,000
│    └─Norm: 2-4                              600
├─EncoderNeuronGroup: 1-2                     --
│    └─EmbedderNeuronGroup: 2-5               --
│    │    └─Linear: 3-7                       60,300
│    │    └─Linear: 3-8                       21,900
│    └─PositionalEncoder: 2-6                 --
│    └─ModuleList: 2-7                        --
│    │    └─EncoderLayer: 3

In [36]:
from torchvision import datasets
IMG_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')

class ClassSpecificImageFolder(datasets.DatasetFolder):
    def __init__(
            self,
            root,
            dropped_classes=[],
            transform = None,
            target_transform = None,
            loader = datasets.folder.default_loader,
            is_valid_file = None,
    ):
        self.dropped_classes = dropped_classes
        super(ClassSpecificImageFolder, self).__init__(root, loader, IMG_EXTENSIONS if is_valid_file is None else None,
                                                       transform=transform,
                                                       target_transform=target_transform,
                                                       is_valid_file=is_valid_file)
        self.imgs = self.samples

    def find_classes(self, directory):
        classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())
        classes = [c for c in classes if c not in self.dropped_classes]
        if not classes:
            raise FileNotFoundError(f"Couldn't find any class folder in {directory}.")

        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx

#from torcheval.metrics.functional import multiclass_confusion_matrix

class CNN(nn.Module):
    def __init__(
        self,
        channels_in,
        nlin="leakyrelu",
        dropout=0.0,
        init_type="uniform",
    ):
        super().__init__()
        # init module list
        self.module_list = nn.ModuleList()
        ### ASSUMES 28x28 image size
        ## compose layer 1
        self.module_list.append(nn.Conv2d(channels_in, 8, 5))
        self.module_list.append(nn.MaxPool2d(2, 2))
        self.module_list.append(self.get_nonlin(nlin))
        # apply dropout
        if dropout > 0:
            self.module_list.append(nn.Dropout(dropout))
        ## compose layer 2
        self.module_list.append(nn.Conv2d(8, 6, 5))
        self.module_list.append(nn.MaxPool2d(2, 2))
        self.module_list.append(self.get_nonlin(nlin))
        ## add dropout
        if dropout > 0:
            self.module_list.append(nn.Dropout(dropout))
        ## compose layer 3
        self.module_list.append(nn.Conv2d(6, 4, 2))
        self.module_list.append(self.get_nonlin(nlin))
        ## add flatten layer
        self.module_list.append(nn.Flatten())
        ## add linear layer 1
        self.module_list.append(nn.Linear(3 * 3 * 4, 20))
        self.module_list.append(self.get_nonlin(nlin))
        ## add dropout
        if dropout > 0:
            self.module_list.append(nn.Dropout(dropout))
        ## add linear layer 1
        self.module_list.append(nn.Linear(20, 10))

        ### initialize weights with se methods
        self.initialize_weights(init_type)

    def initialize_weights(self, init_type):
        # print("initialze model")
        for m in self.module_list:
            if type(m) == nn.Linear or type(m) == nn.Conv2d:
                if init_type == "xavier_uniform":
                    torch.nn.init.xavier_uniform_(m.weight)
                if init_type == "xavier_normal":
                    torch.nn.init.xavier_normal_(m.weight)
                if init_type == "uniform":
                    torch.nn.init.uniform_(m.weight)
                if init_type == "normal":
                    torch.nn.init.normal_(m.weight)
                if init_type == "kaiming_normal":
                    torch.nn.init.kaiming_normal_(m.weight)
                if init_type == "kaiming_uniform":
                    torch.nn.init.kaiming_uniform_(m.weight)
                # set bias to some small non-zero value
                m.bias.data.fill_(0.01)

    def get_nonlin(self, nlin):
        # apply nonlinearity
        if nlin == "leakyrelu":
            return nn.LeakyReLU()
        if nlin == "relu":
            return nn.ReLU()
        if nlin == "tanh":
            return nn.Tanh()
        if nlin == "sigmoid":
            return nn.Sigmoid()
        if nlin == "silu":
            return nn.SiLU()
        if nlin == "gelu":
            return nn.GELU()


    def forward(self, x):
        # forward prop through module_list
        for layer in self.module_list:
            x = layer(x)
        return x

    def forward_activations(self, x):
        # forward prop through module_list
        activations = []
        for layer in self.module_list:
            x = layer(x)
            if (
                isinstance(layer, nn.Tanh)
                or isinstance(layer, nn.Sigmoid)
                or isinstance(layer, nn.ReLU)
                or isinstance(layer, nn.LeakyReLU)
                or isinstance(layer, nn.SiLU)
                or isinstance(layer, nn.GELU)
                or isinstance(layer, ORU)
                or isinstance(layer, ERU)
            ):
                activations.append(x)
        return x, activations
def train(model, trainloader, optimizer, criterion,nb_classes):
    List_mx=[]
    model.train()
    #print('Training')
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in enumerate(trainloader):
        counter += 1
        image, labels = data
        image = image
        labels = labels
        optimizer.zero_grad()
        # forward pass
        outputs = model(image)
        # calculate the loss
        loss = criterion(outputs, labels)
        train_running_loss += loss.item()
        # calculate the accuracy
        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds == labels).sum().item()
        #mx=multiclass_confusion_matrix(preds ,labels,nb_classes,normalize="pred")
        #List_mx.append(mx)
        # backpropagation
        loss.backward()
        # update the optimizer parameters
        optimizer.step()
    
    # loss and accuracy for the complete epoch
    epoch_loss = train_running_loss / counter
    epoch_acc = 100. * (train_running_correct / len(trainloader.dataset))
    return epoch_loss, epoch_acc,List_mx


def validate(model, testloader, criterion,nb_classes):
    List_mx=[]
    model.eval()
    valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    with torch.no_grad():
        for i, data in enumerate(testloader):
            if data is None:  # Skip None values
                continue
            counter += 1
            
            image, labels = data
            image = image
            labels = labels
            # forward pass
            outputs = model(image.to(torch.float32))
            # calculate the loss
            loss = criterion(outputs, labels)
            valid_running_loss += loss.item()
            # calculate the accuracy
            _, preds = torch.max(outputs.data, 1)
            valid_running_correct += (preds == labels).sum().item()
            #mx=multiclass_confusion_matrix(preds ,labels,nb_classes,normalize="pred")
            #List_mx.append(mx)
        
    # loss and accuracy for the complete epoch
    epoch_loss = valid_running_loss / counter
    epoch_acc = 100. * (valid_running_correct / len(testloader.dataset))
    return epoch_loss, epoch_acc,List_mx
def create_frame(step,ax,data):
    ax=ax.cla()
    sns.heatmap(data[step][-1].cpu(),annot=True,cmap="cubehelix",ax=ax,cbar=False)
    plt.title('Epoch {} training {}'.format(step,exp)  )

In [37]:
#L_activations=["gelu","relu","silu","leakyrelu","sigmoid","tanh"]
#csv_files,L_activations

In [38]:
#(2) in https://arxiv.org/pdf/2209.14733.pdf
vec1 = torch.rand(1,2464)
vec2 = torch.rand(1,2464)
class LWLN_loss(nn.Module):
    def __init__(self):
        super(LWLN_loss, self).__init__()
    def forward(self, vec1,vec2):
        loss = (torch.mean((vec1[:,0:208]-vec2[:,0:208])**2)/vec2[:,0:208].std() + 
                 torch.mean((vec1[:,208:1414]-vec2[:,208:1414])**2)/vec2[:,208:1414].std()+ 
                 torch.mean((vec1[:,1414:1514]-vec2[:,1414:1514])**2)/vec2[:,1414:1514].std()+
                 torch.mean((vec1[:,1514:2254]-vec2[:,1514:2254])**2)/vec2[:,1514:2254].std()+
                 torch.mean((vec1[:,2254:2464]-vec2[:,2254:2464])**2)/vec2[:,2254:2464].std())/(6)
        
        return loss
LW=LWLN_loss()
LW(vec1,vec2)

tensor(0.4583)

In [39]:
Cols=["label task 1","index","Accuracy task1",\
      "label task 2","Accuracy task2"]+ \
["weight {}".format(x) for x in range(200)]+["bias {}".format(x) for x in range(200,208)]+ \
["weight {}".format(x) for x in range(208,1408)]+["bias {}".format(x) for x in range(1408,1414)]+ \
["weight {}".format(x) for x in range(1414,1510)]+["bias {}".format(x) for x in range(1510,1514)]+ \
["weight {}".format(x) for x in range(1514,2234)]+["bias {}".format(x) for x in range(2234,2254)]+ \
["weight {}".format(x) for x in range(2254,2454)]+["bias {}".format(x) for x in range(2454,2464)]+ \
["Loader Set","Reconstructed Accuracy ID","Actual Accuracy","Reconstructed Accuracy OOD","Transformer Loss"] 

print(len(Cols))
predicted_Weights= pd.DataFrame(columns=Cols)

# row=["".format(task1),int(ind[0]),ACC[0],"".format(task2),ACC[1]]+vector_aux.to_list()+["train",valid_epoch_acc0,ACC[2],valid_epoch_acc1,L_train[-1]]
# predicted_Weights.append(row, ignore_index=True)
predicted_Weights

2474


,label task 1,index,Accuracy task1,label task 2,Accuracy task2,weight 0,weight 1,weight 2,weight 3,weight 4,...,bias 2459,bias 2460,bias 2461,bias 2462,bias 2463,Loader Set,Reconstructed Accuracy ID,Actual Accuracy,Reconstructed Accuracy OOD,Transformer Loss


# Training 

In [40]:
def optimizer_to(optim, device):
    for param in optim.state.values():
        # Not sure there are any global tensors in the state dict
        if isinstance(param, torch.Tensor):
            param.data = param.data.to(device)
            if param._grad is not None:
                param._grad.data = param._grad.data.to(device)
        elif isinstance(param, dict):
            for subparam in param.values():
                if isinstance(subparam, torch.Tensor):
                    subparam.data = subparam.data.to(device)
                    if subparam._grad is not None:
                        subparam._grad.data = subparam._grad.data.to(device)
def scheduler_to(sched, device):
    for param in sched.__dict__.values():
        if isinstance(param, torch.Tensor):
            param.data = param.data.to(device)
            if param._grad is not None:
                param._grad.data = param._grad.data.to(device)

In [41]:
import datetime
print(datetime.datetime.now())

2024-05-15 17:40:46.033531


In [42]:
mod

TransformerAE(
  (enc1): EncoderNeuronGroup(
    (embed): EmbedderNeuronGroup(
      (neuron_l1): Linear(in_features=200, out_features=300, bias=True)
      (neuron_l2): Linear(in_features=72, out_features=300, bias=True)
    )
    (pe): PositionalEncoder()
    (layers): ModuleList(
      (0-3): 4 x EncoderLayer(
        (norm_1): Norm()
        (norm_2): Norm()
        (attn): MultiHeadAttention(
          (q_linear): Linear(in_features=300, out_features=300, bias=True)
          (v_linear): Linear(in_features=300, out_features=300, bias=True)
          (k_linear): Linear(in_features=300, out_features=300, bias=True)
          (dropout): Dropout(p=0.12, inplace=False)
          (out): Linear(in_features=300, out_features=300, bias=True)
        )
        (ff): FeedForward(
          (linear_1): Linear(in_features=300, out_features=300, bias=True)
          (dropout): Dropout(p=0.12, inplace=False)
          (linear_2): Linear(in_features=300, out_features=300, bias=True)
        )
   

In [46]:
import random
import csv 
import time
import sys
import time
import datetime
import warnings
import traceback


warnings.filterwarnings("ignore")
results_path="./mixed model/"

track=0
lr=0.01
criterion = nn.MSELoss()
LW=LWLN_loss()

optimizer = Adam(mod.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-8, max_lr=0.005, step_size_up=256, mode="triangular2", cycle_momentum=False)
num_epochs=80
#nb_batches=2800



Cols=["label task 1","index","Accuracy task1",\
      "label task 2","Accuracy task2"]+ \
["weight {}".format(x) for x in range(200)]+["bias {}".format(x) for x in range(200,208)]+ \
["weight {}".format(x) for x in range(208,1408)]+["bias {}".format(x) for x in range(1408,1414)]+ \
["weight {}".format(x) for x in range(1414,1510)]+["bias {}".format(x) for x in range(1510,1514)]+ \
["weight {}".format(x) for x in range(1514,2234)]+["bias {}".format(x) for x in range(2234,2254)]+ \
["weight {}".format(x) for x in range(2254,2454)]+["bias {}".format(x) for x in range(2454,2464)]+ \
["Loader Set","Reconstructed Accuracy ID","Actual Accuracy","Reconstructed Accuracy OOD","Transformer Loss","lr",'epochCNN','ActivationCNN'] 


torch.save({'epoch':-1,'model_state_dict': mod.state_dict(),'optimizer_state_dict': optimizer.state_dict(),},results_path+'AE epoch {}.pth'.format(-1))
batch_size=1024
cs_tr=CustomDataset(train_pair,batch_size=batch_size)
nb_batches = len(cs_tr)//batch_size

optimizer_to(optimizer,device)
scheduler_to(scheduler,device)
mod.train()
for epoch in range(num_epochs):
    start_time_epoch = time.time()
    for i in tqdm(range(nb_batches)):
        #start_time_batch = time.time()
        loss_tr=torch.tensor(0.0, device=device)
        
        Dataset,EXP,ACC,U = cs_tr[i]
        x1,x2,tg = Dataset[:,0,:], Dataset[:,1,:],Dataset[:,2,:]
        try:
            x1=x1.to(torch.float16)
            x2=x2.to(torch.float16)
            tg=tg.to(torch.float16)
            
            x1=x1.to(device)
            x2=x2.to(device)
            tg=tg.to(device)
            mod=mod.to(device)

            optimizer.zero_grad()

            output = mod(x1,x2)
            
            loss_tr = 0.5*criterion(output[0],tg) +0.5*LW(output[0],tg)

        except Exception as e:
            # Print the exception
            print("An exception occurred:", e)
            traceback.print_exc()
            # Continue with the loop
            continue
        
        loss_tr=loss_tr.detach()
        loss_tr.requires_grad=True
        loss_tr.backward()
        optimizer.step()
        scheduler.step()

        end_time_batch = time.time()
        #execution_time = end_time_batch - start_time_batch
        #print("Batch Execution time:", execution_time, "seconds")
        torch.cuda.empty_cache()
        
    end_time_epoch = time.time()
    execution_time = end_time_epoch - start_time_epoch
    print("Epoch Execution time:", execution_time, "seconds")

        

    if epoch%30==0 or (epoch in [1,3,5,10]):
        for vect in tqdm(range(len(x1))):
            y_pred=torch.unsqueeze(output[0][vect], 0) 
            y =torch.unsqueeze(tg[vect], 0) 
            
            selected_row = cs_tr.df.iloc[int(U[vect][0]), 11:17]  
            columns_with_one = selected_row[selected_row == 1].index.tolist()
            activ=columns_with_one
            epochCNN=cs_tr.df.loc[int(U[vect][0])]['epoch']
            
            
            checkpoint=OrderedDict()
            vector_aux= output[0][vect].detach()
            y_pred=vector_aux.cpu()

            task1=[int(x) for x in EXP[vect][0]]
            task2=[int(x) for x in EXP[vect][1]]
            task3=sorted(task1+task2)


            All=list(range(10))
            L2=[k for k in All if k not in task3] #Classes to test on (In distribution)
            L_others=[k for k in All if k not in task3] # Out of distribution classes

            checkpoint["module_list.0.weight"]=torch.tensor(np.array(y_pred[0:200]).reshape([8, 1, 5, 5]))
            checkpoint["module_list.0.bias"]=torch.tensor(np.array(y_pred[200:208]).reshape([8]))

            checkpoint["module_list.3.weight"]=torch.tensor(np.array(y_pred[208:1408]).reshape([6, 8, 5, 5]))
            checkpoint["module_list.3.bias"]=torch.tensor(np.array(y_pred[1408:1414]).reshape([6]))

            checkpoint["module_list.6.weight"]=torch.tensor(np.array(y_pred[1414:1510]).reshape([4, 6, 2, 2]))
            checkpoint["module_list.6.bias"]=torch.tensor(np.array(y_pred[1510:1514]).reshape([4]))

            checkpoint["module_list.9.weight"]=torch.tensor(np.array(y_pred[1514:2234]).reshape([20,36]))
            checkpoint["module_list.9.bias"]=torch.tensor(np.array(y_pred[2234:2254]).reshape([20]))

            checkpoint["module_list.11.weight"]=torch.tensor(np.array(y_pred[2254:2454]).reshape([10,20]))
            checkpoint["module_list.11.bias"]=torch.tensor(np.array(y_pred[2454:2464]).reshape([10]))

            Brain = CNN(1,activ[0],0,"kaiming_uniform")

            model=copy.deepcopy(Brain).to(torch.float32)
            model.load_state_dict(checkpoint)

            criterion_CNN0=CrossEntropyLoss()

            test_IF0=ClassSpecificImageFolder( root="./data/SplitMnist/test/",dropped_classes=[str(x) for x in L2],transform=transforms.Compose([ transforms.ToTensor(),transforms.Grayscale(1)]))
            Ts_DL0 = DataLoader(dataset=test_IF0, batch_size=400, num_workers=0, shuffle=True)
            
            _, valid_epoch_acc0,_= validate(model, Ts_DL0,  criterion_CNN0,10)
            if len(task3)==10:
                valid_epoch_acc1=valid_epoch_acc0
                continue
            else:
                criterion_CNN1=CrossEntropyLoss()
                test_IF1=ClassSpecificImageFolder( root="./data/SplitMnist/test/",dropped_classes=[str(x) for x in task3],transform=transforms.Compose([ transforms.ToTensor(),transforms.Grayscale(1)]))
                Ts_DL1 = DataLoader(dataset=test_IF1, batch_size=400, num_workers=0, shuffle=True)
            
            valid_epoch_loss0, valid_epoch_acc1,L_mx= validate(model, Ts_DL1,  criterion_CNN1,10)

            lr = optimizer.param_groups[0]["lr"]

            
            
            vectstring=f"{y_pred.tolist()}".replace(" ", "")[1:-1].split(',')
            row_part=f"{task1};{int(U[vect][0])};{float(ACC[vect][0])};{task2};{float(ACC[vect][1])}".split(";")
            row_part2=f"train,{valid_epoch_acc0},{float(ACC[vect][2])},{valid_epoch_acc1},{float(loss_tr.detach().cpu())},{lr},{epochCNN},{activ[0]}".split(",")
            row=row_part+vectstring+row_part2
            
            temp_file = os.path.join(results_path,f'AE Tracking {track}.csv')
        
            if vect%100==0:
                print(task3,len(Ts_DL0.dataset))
                print(L2,len(Ts_DL1.dataset))
                print("epoch" ,epoch, "\t", "\t batch: ",f"/{len(Dataset)}",f"\t Learning rate :" ,f"{lr:.8f}")
                temp_file = os.path.join(results_path, f'AE Tracking {track}.csv')
                with open(temp_file, mode='a', newline='\n') as file:
                    writer = csv.writer(file)
                    writer.writerow(Cols)
                
            with open(temp_file, mode='a', newline='\n') as file:
                writer = csv.writer(file)
                writer.writerow(row)
            if vect%100==0:
                track += 1

        csv_files = [file for file in os.listdir(results_path) if file.endswith(".csv")]
        csv_files.sort(key=lambda x: int(x.split(" ")[2][:-4]))
        
        df = pd.read_csv(os.path.join(results_path, "AE Tracking 0.csv"),header=[0])
        res= pd.DataFrame(columns=df.columns)
        for file in csv_files:
            df = pd.read_csv(os.path.join(results_path, file), header=[0])
            res = res.append(df, ignore_index=True)
        print("epoch",epoch,'\n','worst_CNN_ACC \t',  float(min(res[["Reconstructed Accuracy ID"]].values)),'\n','best_CNN_ACC \t',float(max(res[["Reconstructed Accuracy ID"]].values)),'\n','max_Transformer_loss \t',float(max(res[["Transformer Loss"]].values)),'\n','min_CNN_ACC \t',float(min(res[["Transformer Loss"]].values)))
        torch.save({'epoch':epoch,'model_state_dict': mod.state_dict(),'optimizer_state_dict': optimizer.state_dict(),'Batch Loss':loss_tr.detach().cpu().item(),'last_batch_CNN_ACC':valid_epoch_acc0,'worst_CNN_ACC':  float(min(res[["Reconstructed Accuracy ID"]].values)),'best_CNN_ACC':float(max(res[["Reconstructed Accuracy ID"]].values)),'max_Transformer_loss':float(max(res[["Transformer Loss"]].values)),'min_CNN_ACC':float(min(res[["Transformer Loss"]].values))},results_path+'AE epoch {}.pth'.format(epoch))


 28%|██▊       | 164/581 [19:55<51:10,  7.36s/it] 

In [ ]:
print(x1.shape,x2.shape,tg.shape)
vector_aux.shape, len(x1) ,vector_aux[vect,5:200].shape#, vector_aux[p,0:200].shape

In [ ]:
loss_tr

In [25]:
torch.save({'epoch':epoch,'model_state_dict': mod.state_dict(),'optimizer_state_dict': optimizer.state_dict(),'Mixed Loss':batch_loss.detach().cpu().item(),'CNN_ACC':valid_epoch_acc0},'./model/AE epoch {}.pth'.format(epoch))

In [20]:
batch_loss.requires_grad=True
batch_loss,nb_samples

(tensor(47223.6875, requires_grad=True), 21868)

In [19]:
len(ACC)

80

In [21]:
#https://discuss.pytorch.org/t/memory-management-using-pytorch-cuda-alloc-conf/157850
#https://stackoverflow.com/questions/73747731/runtimeerror-cuda-out-of-memory-how-can-i-set-max-split-size-mb

import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                           EXP: 488.2 KiB
                           ACC: 488.2 KiB
                             U: 488.2 KiB
                     Pairs_exp: 190.1 KiB
                    train_pair: 133.4 KiB
                     train_tgt: 133.4 KiB
                     test_pair: 36.3 KiB
                      test_tgt: 36.3 KiB
                    vectstring: 20.1 KiB
                      val_pair: 20.1 KiB


In [26]:
pair[0],task1

([0, 6, 7, 8, 9], [0, 2, 3, 5, 8])

In [24]:
predicted_Weights

,label task 1,index,Accuracy task1,label task 2,Accuracy task2,weight 0,weight 1,weight 2,weight 3,weight 4,...,bias 2462,bias 2463,Loader Set,Reconstructed Accuracy ID,Actual Accuracy,Reconstructed Accuracy OOD,Transformer Loss,lr,epochCNN,ActivationCNN
0,,15266,98.333333,,98.093333,1.693639,-0.393555,0.068379,0.583934,-1.534439,...,-1.608331,0.621260,train,8.38,98.250000,8.38,2.000025,0.005,16,leakyrelu
1,,15266,98.333333,,97.883333,2.080894,-0.042077,0.173723,0.623434,-1.614910,...,-1.388891,0.539818,train,7.79,97.948333,7.79,1.986340,0.005,16,leakyrelu
2,,15266,98.333333,,98.410000,2.049980,-0.242580,0.318586,0.646433,-1.770652,...,-1.287083,0.398330,train,9.31,98.070000,9.31,2.011771,0.005,16,leakyrelu
3,,15266,98.333333,,98.286667,1.815176,-0.604665,-0.257339,0.312119,-1.143257,...,-1.217142,0.706113,train,9.93,97.825000,9.93,2.031300,0.005,16,leakyrelu
4,,15266,98.333333,,98.236667,1.697706,-0.546888,0.326315,0.452815,-1.507497,...,-1.418500,0.340476,train,10.97,98.133333,10.97,2.037839,0.005,16,leakyrelu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,,15221,97.201667,,98.041667,1.963520,-0.738335,-0.012039,0.313362,-1.397939,...,-1.352349,0.799487,train,9.55,98.003333,9.55,2.002596,0.005,16,leakyrelu
116,,20039,97.770000,,97.316667,1.746097,-0.815134,-0.120588,0.589807,-1.498475,...,-0.946741,-0.098292,train,9.70,97.821667,9.70,1.936134,0.005,16,leakyrelu
117,,20039,97.770000,,97.650000,1.846137,-0.439110,0.093636,0.326424,-1.411710,...,-0.701774,0.514087,train,8.21,96.220000,8.21,1.968303,0.005,16,leakyrelu
118,,20039,97.770000,,97.646667,1.797259,-0.159525,-0.572254,0.156177,-1.160839,...,-1.177182,0.417260,train,8.78,97.841667,8.78,1.933440,0.005,16,leakyrelu


In [ ]:
        # if o%5==0 and p<5:
        #     print("\t",[int(x) for x in labels[0]],"\t",[int(x) for x in labels[1]],f"\t {loss_tr.item():.5f}" ,"\t Epoch ",epoch,"/300",path[-11:-4],activ)
        # L_train.append(loss_tr.item())
        
        for state in optimizer.state.values():
            for k, v in state.items():
                if isinstance(v, torch.Tensor):
                    state[k] = v.cuda()
        lr = optimizer.param_groups[0]["lr"]
        if i%5==0:
            print("\t",state['step'] , f"\t Learning rate :" ,f"{lr:.8f}","\t min loss in batch: ", f"{min(L_train[-120:]):.7f}" , "\t avg loss in batch" ,f"{sum(L_train[-120:])/len(L_train[-120:]):.7f}" )

            for muk in range(len(test_pair)//80):

                start_index = muk * 80
                end_index = min((muk + 1) * 80, len(test_pair))
                print(start_index, end_index)
                cs_ts=CustomDataset(test_pair,start_index,end_index,path,activ)
                for x1,x2,tg,labels in DataLoader(cs_ts):
                    x1=x1.to(torch.float32)
                    x2=x2.to(torch.float32)
                    tg=tg.to(torch.float32)
                    
                    x1=x1.to(device)
                    x2=x2.to(device)
                    tg=tg.to(device)
                    mod=mod.to(device)
                    
                    mod.eval()
                    output = mod(x1,x2)
                    loss_ts = criterion(output[0],tg)
                    print("Test--",[int(x) for x in labels[0]],[int(x) for x in labels[1]],f"{loss_ts.item():.5f}" ,"Epoch ",epoch,"/300",path,activ)
                    L_test.append(loss_ts.item())
            for muk in range(len(val_pair)//80):
                start_index = muk * 80
                end_index = min((muk + 1) * 80, len(val_pair))
                cs_val=CustomDataset(val_pair,start_index ,end_index,path,activ)

                for x1,x2,tg,labels in DataLoader(cs_val):
                    x1=x1.to(torch.float32)
                    x2=x2.to(torch.float32)
                    tg=tg.to(torch.float32)
                    
                    x1=x1.to(device)
                    x2=x2.to(device)
                    tg=tg.to(device)
                    mod=mod.to(device)
                    
                    mod.eval()
                    optimizer.zero_grad()
                    output = mod(x1,x2)
                    loss_val = criterion(output[0],tg)
                    print("Validation--",[int(x) for x in labels[0]],[int(x) for x in labels[1]],"MAE ",f"{loss_val.item():.5f}","Epoch ",epoch,"/300",path,activ)
                    L_val.append(loss_val.item())
        
        
        
        np.save(f'Train epoch{epoch}.npy',np.array(L_train))
        np.save(f'Val epoch{epoch}.npy', np.array(L_val))
        np.save(f'Test epoch{epoch}.npy', np.array(L_test))
        Lt_epoch.append(np.array(L_train).mean())
        Ls_epoch.append(np.array(L_test).mean())
        Lv_epoch.append(np.array(L_val).mean())
        if epoch%2==0:
            torch.save({'epoch':epoch,'model_state_dict': mod.state_dict(),'optimizer_state_dict': optimizer.state_dict(),},'./model/AE epoch {} {}.pth'.format(epoch,path[-11:-4]))

In [ ]:
# Define the file path to your checkpoint
checkpoint_path = ".//model//AE epoch 0 epoch36.pth"

# Load the checkpoint
checkpoint = torch.load(checkpoint_path,pickle_module=dill,map_location=device)

# Create your model instance
model = TransformerAE(max_seq_len=176,
                    N=4,
                    heads=4,
                    d_model=700,
                    d_ff=700,
                    neck=200,
                    dropout=0.1
                   )

# Load the model state_dict
model.load_state_dict(checkpoint['model_state_dict'])

# Load other components as needed
# For example, optimizer state_dict, epoch, loss, etc.
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']

# Put the model in evaluation mode
model.eval()

criterion = nn.MSELoss()

#len(test_pair),len(train_pair),len(val_pair)
for epoch in tqdm(range(num_epochs)):
    if epoch==0:
        L_train=[]
        L_val=[]
        L_test=[]
        for path in csv_files:

            for activ in L_activations:
                for muk in range(len(test_pair)//800):

                    start_index = muk * 800
                    end_index = min((muk + 1) * 800, len(test_pair))
                    print(start_index, end_index)
                    cs_ts=CustomDataset(test_pair,start_index,end_index,path,activ)
                    for x1,x2,tg,labels in DataLoader(cs_ts):
                        x1=x1.to(torch.float32)
                        x2=x2.to(torch.float32)
                        tg=tg.to(torch.float32)

                        x1=x1.to(device)
                        x2=x2.to(device)
                        tg=tg.to(device)
                        mod=mod.to(device)

                        mod.eval()
                        output = mod(x1,x2)
                        loss_ts = criterion(output[0],tg)

                        L_test.append(loss_ts.item())
                    print("Test--",[int(x) for x in labels[0]],[int(x) for x in labels[1]],f"{loss_ts.item():.5f}" ,sum(L_test[-800:])/len(L_test[-800:]))
                for muk in range(len(val_pair)//800):
                    start_index = muk * 800
                    end_index = min((muk + 1) * 800, len(val_pair))
                    cs_val=CustomDataset(val_pair,start_index ,end_index,path,activ)

                    for x1,x2,tg,labels in DataLoader(cs_val):
                        x1=x1.to(torch.float32)
                        x2=x2.to(torch.float32)
                        tg=tg.to(torch.float32)

                        x1=x1.to(device)
                        x2=x2.to(device)
                        tg=tg.to(device)
                        mod=mod.to(device)

                        mod.eval()
                        optimizer.zero_grad()
                        output = mod(x1,x2)
                        loss_val = criterion(output[0],tg)
                        L_val.append(loss_val.item())
                    

# reconstruct Checkpoint

In [ ]:
output[0].shape ,activ,task3,"kaiming_uniform"

In [ ]:

checkpoint=OrderedDict()
vector_aux= output[0].detach()
vector_aux=vector_aux.cpu()

task1=[int(x) for x in labels[0]]
task2=[int(x) for x in labels[1]]
task3=sorted(task1+task2)


All=list(range(10))
L2=[k for k in All if k not in task3]
L_others=[k for k in All if k not in L2]

checkpoint["module_list.0.weight"]=torch.tensor(np.array(vector_aux[:,0:200]).reshape([8, 1, 5, 5]))
checkpoint["module_list.0.bias"]=torch.tensor(np.array(vector_aux[:,200:208]).reshape([8]))

checkpoint["module_list.3.weight"]=torch.tensor(np.array(vector_aux[:,208:1408]).reshape([6, 8, 5, 5]))
checkpoint["module_list.3.bias"]=torch.tensor(np.array(vector_aux[:,1408:1414]).reshape([6]))

checkpoint["module_list.6.weight"]=torch.tensor(np.array(vector_aux[:,1414:1510]).reshape([4, 6, 2, 2]))
checkpoint["module_list.6.bias"]=torch.tensor(np.array(vector_aux[:,1510:1514]).reshape([4]))

checkpoint["module_list.9.weight"]=torch.tensor(np.array(vector_aux[:,1514:2234]).reshape([20,36]))
checkpoint["module_list.9.bias"]=torch.tensor(np.array(vector_aux[:,2234:2254]).reshape([20]))

checkpoint["module_list.11.weight"]=torch.tensor(np.array(vector_aux[:,2254:2454]).reshape([10,20]))
checkpoint["module_list.11.bias"]=torch.tensor(np.array(vector_aux[:,2454:2464]).reshape([10]))

model = CNN(1,activ,0,"kaiming_uniform")
model.load_state_dict(checkpoint)
model.eval()

criterion_CNN=CrossEntropyLoss()

test_IF=ClassSpecificImageFolder( root="./data/SplitMnist/test/",dropped_classes=L2,transform=transforms.Compose([ transforms.ToTensor(),transforms.Grayscale(1)]))
Ts_DL = DataLoader(dataset=test_IF, batch_size=90, num_workers=0, shuffle=True)
valid_epoch_loss0, valid_epoch_acc0,L_mx= validate(model, Ts_DL,  criterion_CNN,10)
print(valid_epoch_loss0, valid_epoch_acc0)

test_IF=ClassSpecificImageFolder( root="./data/SplitMnist/test/",dropped_classes=L_others,transform=transforms.Compose([ transforms.ToTensor(),transforms.Grayscale(1)]))
Ts_DL = DataLoader(dataset=test_IF, batch_size=90, num_workers=0, shuffle=True)
valid_epoch_loss0, valid_epoch_acc0,L_mx= validate(model, Ts_DL,  criterion_CNN,10)
print(valid_epoch_loss0, valid_epoch_acc0)

#if not(os.path.isdir('./checkpoints/')):
#    os.mkdir('./checkpoints/')
#torch.save(model.state_dict(), './checkpoints/Cumulative/{}/{}/{}/1/Reconsturcted checkpoint.pth'.format(ListExperiences[class_order_idx][:5],list(i)[0],list(i)[1]))

In [ ]:
vector_aux[0:200].shape

In [ ]:
#per epoch:
    #Checkpoint AE
    #dataframe of weights + respective performance
    #dataframe 